# OpenScale Configuration

This notebook 
1. creates a dummy subscription
2. for the dummy subscription, creates and configures a custom monitor

In [1]:
import os
import json
from datetime import datetime, timezone, timedelta
import time
import uuid
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import WMLCredentialsCP4D
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import wos_sdk_utils as wos_util
import wml_sdk_utils as wml_util

In [2]:
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
SUBSCRIPTION_NAME = "cifar monitor wendy"

WML_SPACE_ID = '934341dc-0a71-4d86-9c09-a47261359cca' 

WOS_GUID = '00000000-0000-0000-0000-000000000000'

In [3]:
wos_client = wos_util.get_client()
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)

## 1. Create Subscription
Here we create a dummy subscription that does not link to a real deployment endpoint.

In [4]:
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest

In [5]:
existing_providers_dict = wos_client.service_providers.list().get_result().to_dict()['service_providers']
existing_providers = [sp['entity']['name'] for sp in existing_providers_dict]

SERVICE_PROVIDER_ID = next((sp['metadata']['id'] for sp in existing_providers_dict \
                            if sp['entity']['name'] == SERVICE_PROVIDER_NAME))
print(f"Service provider ID: {SERVICE_PROVIDER_ID}")

Service provider ID: 42431cc1-115a-49b8-b6b7-b73527804739


In [6]:
wos_client.service_providers.show()

None,active,OpenScale Headless Service Provider,custom_machine_learning,2022-05-03 14:45:46.933000+00:00,42431cc1-115a-49b8-b6b7-b73527804739
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2022-01-13 06:19:31.996000+00:00,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6


In [7]:
existing_subscriptions_dict = wos_client.subscriptions.list().get_result().to_dict()['subscriptions']
existing_subscriptions = [sp['entity']['asset']['name'] for sp in existing_subscriptions_dict]

if not SUBSCRIPTION_NAME in existing_subscriptions:
    
    # generate dummy information
    ASSET_ID = str(uuid.uuid4())
    ASSET_NAME = SUBSCRIPTION_NAME
    url = ''

    ASSET_DEPLOYMENT_ID = str(uuid.uuid4())
    ASSET_DEPLOYMENT_NAME = SUBSCRIPTION_NAME
    
    # pass dummy information to create a dummy subscription
    subscription_details = wos_client.subscriptions.add(
        data_mart_id=WOS_GUID,
        service_provider_id=SERVICE_PROVIDER_ID,
        asset=Asset(
            asset_id=ASSET_ID,
            name=ASSET_NAME,
            url=url,
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.MULTICLASS_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=ASSET_DEPLOYMENT_ID,
            name=ASSET_DEPLOYMENT_NAME,
            deployment_type= DeploymentTypes.ONLINE
        ),
        asset_properties=AssetPropertiesRequest(
            probability_fields=['probability']
            )
    ).result
    
    SUBSCRIPTION_ID = subscription_details.metadata.id
    print("Subscription ID: {}".format(SUBSCRIPTION_ID))
else:
    SUBSCRIPTION_ID = next((sp['metadata']['id'] for sp in existing_subscriptions_dict \
                            if sp['entity']['asset']['name'] == SUBSCRIPTION_NAME))
    print("Subscription ID: {}".format(SUBSCRIPTION_ID))

Subscription ID: 0ed3bd5b-6e13-4863-95d7-82ba35900285


In [8]:
wos_client.subscriptions.show()

291c22ee-90cd-43c8-bbac-095ee2cbd429,cifar monitor wendy,00000000-0000-0000-0000-000000000000,fd2ca910-2c64-4bc6-98ae-6546665fe75a,cifar monitor wendy,42431cc1-115a-49b8-b6b7-b73527804739,active,2022-05-03 20:27:31.457000+00:00,0ed3bd5b-6e13-4863-95d7-82ba35900285
6eb7d805-d311-4bc3-878d-5d43edb76ad3,GermanCreditRiskModelPreProdICP,00000000-0000-0000-0000-000000000000,5549aabc-ca61-46ce-ab8a-d7d33cc1b1d5,GermanCreditRiskModelPreProdICP,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6,active,2022-01-13 06:23:13.902000+00:00,3944c36d-c97d-4cbe-9066-d9637f4ec715
a6e676d0-c9c3-4c7a-b1b3-5fdf10e2b22d,GermanCreditRiskModelChallengerICP,00000000-0000-0000-0000-000000000000,641e28cf-ea00-4867-9562-5cb7bfaa281c,GermanCreditRiskModelChallengerICP,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6,active,2022-01-13 06:21:41.540000+00:00,ec94173d-6cfa-4f47-ae04-5fb1497710f5


## 2. Configure Custom Metric Monitors

### Check existence of custom monitor instance

In [9]:
monitor_id = 'dummy_monitor_example_wendy'

In [10]:
existing_monitor_instance = wos_util.get_monitor_instance(monitor_id,SUBSCRIPTION_ID,wos_client)
existing_monitor_instance

No existing instance for monitor dummy_monitor_example_wendy found with subscription 0ed3bd5b-6e13-4863-95d7-82ba35900285


### Create one custom monitor instance per monitor

In [11]:
integrated_system_id = '9e9ed5ba-b462-422c-b9cb-cb3becbdfb9d'
custom_metrics_wait_time = 360 # time in seconds 

In [12]:
# If it does not exist, then create one
if existing_monitor_instance is None:
    target = Target(
            target_type=TargetTypes.SUBSCRIPTION,
            target_id=SUBSCRIPTION_ID
        )
    parameters = {
        "custom_metrics_provider_id": integrated_system_id,
        "custom_metrics_wait_time":   custom_metrics_wait_time 
    }

    # Update the threshold for metrics in your custom monitor metric
    thresholds = [ MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=100),
                   MetricThresholdOverride(metric_id='specificity', type = MetricThresholdTypes.LOWER_LIMIT, value=800)]

    # create the custom monitor instance id here.
    custom_monitor_instance_details = wos_client.monitor_instances.create(
                data_mart_id=WOS_GUID,
                background_mode=False,
                monitor_definition_id=monitor_id,
                target=target,
                parameters=parameters,
                thresholds = thresholds
    ).result
else:
    pass

custom_monitor_instance_details




 Waiting for end of monitor instance creation 97fbef2e-d8a7-4a9e-8ac8-7bbc0e6ae2db 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Manual Evaluation

In [13]:
import uuid
from pprint import pprint

subscription_id = SUBSCRIPTION_ID
wml_deployment_id = 'af5ea879-60b8-4f9e-a276-893e1f986a34'

print('*'*30,monitor_id,'*'*30)
parameters = {
    "custom_metrics_provider_id": integrated_system_id,
    "custom_metrics_wait_time":   custom_metrics_wait_time,
    "run_details": {
    "run_id": str(uuid.uuid4()),
    "run_status": "Running"
    }
}

payload= {
    "data_mart_id" : WOS_GUID,
    "subscription_id" : subscription_id,
    "custom_monitor_id" : monitor_id,
    "custom_monitor_instance_id" : custom_monitor_instance_details.metadata.id,
    "custom_monitor_instance_params": parameters

}

input_data= { "input_data": [ { "values": payload } ]
            }

#     print(input_data)

job_details = wml_client.deployments.score(wml_deployment_id, input_data)
pprint(job_details)

****************************** dummy_monitor_example_wendy ******************************
{'predictions': [{'values': [[{'measurement_id': 'd99f1c28-d046-46b9-826c-1b7d0f57e439',
                               'metrics': [{'sensitivity': 0.85,
                                            'specificity': 1.2}],
                               'run_id': 'dcaa6b1d-28d4-4737-82c1-c7ba7a332ca5',
                               'timestamp': '2022-05-03T21:13:58.014515Z'}]]}]}
